<a href="https://colab.research.google.com/github/speedofsound98/Cave_Id_project/blob/main/FinalProject.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install transformers torch faiss-cpu pillow tqdm matplotlib opencv-python-headless

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 36.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 37.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 38.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 3.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 6.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 11.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 6.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 73.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 30.7/30.7 MB 51.8 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstallin

In [ ]:
from transformers import AutoImageProcessor, Dinov2ForImageClassification
import torch
import faiss
import numpy as np
from PIL import Image
import os
import cv2
from tqdm.notebook import tqdm
import matplotlib.pyplot as plt
import json


In [ ]:
# Load the pre-trained model and image processor
processor = AutoImageProcessor.from_pretrained("facebook/dinov2-base")
model = Dinov2ForImageClassification.from_pretrained("facebook/dinov2-base")

# Set the model to evaluation mode
model.eval()


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


preprocessor_config.json:   0%|          | 0.00/436 [00:00<?, ?B/s]

Using a slow image processor as `use_fast` is unset and a slow processor was saved with this model. `use_fast=True` will be the default behavior in v4.48, even if the model was saved with a slow processor. This will result in minor differences in outputs. You'll still be able to use a slow processor with `use_fast=False`.


config.json:   0%|          | 0.00/548 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/346M [00:00<?, ?B/s]

Some weights of Dinov2ForImageClassification were not initialized from the model checkpoint at facebook/dinov2-base and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Dinov2ForImageClassification(
  (dinov2): Dinov2Model(
    (embeddings): Dinov2Embeddings(
      (patch_embeddings): Dinov2PatchEmbeddings(
        (projection): Conv2d(3, 768, kernel_size=(14, 14), stride=(14, 14))
      )
      (dropout): Dropout(p=0.0, inplace=False)
    )
    (encoder): Dinov2Encoder(
      (layer): ModuleList(
        (0-11): 12 x Dinov2Layer(
          (norm1): LayerNorm((768,), eps=1e-06, elementwise_affine=True)
          (attention): Dinov2SdpaAttention(
            (attention): Dinov2SdpaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.0, inplace=False)
            )
            (output): Dinov2SelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.0, inplace=False)

In [ ]:
# from google.colab import files
# uploaded = files.upload()

from google.colab import drive
drive.mount('/content/drive')
TESTSET_PATH = "/content/drive/MyDrive/cavestesting/"
DATASET_PATH = "/content/drive/MyDrive/caves_open_MVP/"


Mounted at /content/drive


In [ ]:
def create_test_index(dataset_path: str) -> tuple:
    """
    Create an FAISS index for images in the specified path.
    Works for both dataset and test set.
    """
    index = faiss.IndexFlatL2(768)
    image_to_cave = {}
    all_embeddings = {}

    image_id = 0

    with torch.no_grad():
        for root, dirs, files in os.walk(dataset_path):
            for img_file in files:
                img_path = os.path.join(root, img_file)

                # Skip non-image files
                if not img_file.lower().endswith(('.jpg', '.jpeg', '.png')):
                    continue

                try:
                    image_tensor = load_image(img_path).to(device)
                    embedding = model(image_tensor).last_hidden_state.mean(dim=1).cpu().numpy()

                    # Check for NaN or invalid embeddings
                    if np.isnan(embedding).any() or embedding.shape != (1, 768):
                        continue

                    index.add(embedding)

                    # Map image to the parent folder name (cave name)
                    relative_path = os.path.relpath(img_path, dataset_path)
                    cave_name = relative_path.split(os.sep)[0]
                    image_to_cave[image_id] = cave_name

                    index.add(embedding)
                    image_to_cave[image_id] = cave_name  # Map index to cave label
                    all_embeddings[img_path] = embedding.tolist()  # Save embedding
                    image_id += 1

                except Exception as e:
                    print(f"Error processing {img_path}: {e}")
                    continue


    with open("test_image_to_cave.json", "w", encoding="utf-8") as f:
        json.dump(image_to_cave, f, ensure_ascii=False, indent=4)

    with open("all_test_embeddings.json", "w", encoding="utf-8") as f:
        json.dump(all_embeddings, f, ensure_ascii=False, indent=4)

    # faiss.write_index(index, "cave_index.bin")

    faiss.write_index(index, f"index_{os.path.basename(dataset_path)}.bin")

    return index, image_to_cave

test_index, test_image_to_cave = create_test_index(TESTSET_PATH)



In [ ]:
print(test_image_to_cave)

{0: 'masha_tarwill_1.JPG', 1: 'hamalkodet_1.jpg', 2: 'hamalkodet_2.jpg', 3: 'Um_kalaa_2.jpg', 4: 'otzar_1.JPG', 5: 'arugot_1.jpg', 6: 'etzba_1.jpg', 7: 'ashalim_1.JPG', 8: 'golgolet_1.JPG', 9: 'golgolet_2.JPG', 10: 'dir_kadis_1.JPG', 11: 'haigrot_1.jpg', 12: 'haamonit_1.JPG', 13: 'hagai_1.jpg', 14: 'haner_1.JPG', 15: 'chatrurim_1.JPG', 16: 'miglash_1.JPG', 17: 'nahal_natuf_1.JPG', 18: 'namer_1.jpg', 19: 'nekev_ukafim_1.JPG', 20: 'atarot_1.JPG', 21: 'arak_a_naasna_1.JPG', 22: 'kirkas_1.JPG', 23: 'shukba_1.JPG', 24: 'shen_ramon_1.jpg', 25: 'serach_1.JPG'}


In [ ]:
def get_image_embedding(image_path):
    # Open and preprocess the image
    img = Image.open(image_path)
    inputs = processor(images=img, return_tensors="pt")

    # Get the image embeddings from the model
    with torch.no_grad():
        features = model.forward(**inputs)

    return features[0].numpy()  # Extract the features as a numpy array

# Example: Load the query image
query_image_path = 'DSCN1824.JPG'  # Update with your query image
query_embedding1 = get_image_embedding(query_image_path)
#  query_embedding2 = get_image_embedding("pic2.jpg")
#  query_embedding3 = get_image_embedding("David-Ben-Gurion-1.jpg")
print(query_embedding1)
# print(query_embedding2)
# print(query_embedding3)


[[[-0.9047337   0.85604507  2.8009245  ... -1.0853302  -0.06766028
   -0.9890508 ]
  [-2.3482482  -1.3050892   0.09322477 ... -0.13829899  0.2115592
   -3.2923145 ]
  [-2.611753    0.8117259   0.23976222 ... -1.5345615   0.22542712
   -1.2668518 ]
  ...
  [ 0.80799675  1.6683966   0.34996396 ...  2.5491183  -1.6863939
   -0.76994914]
  [ 0.4597337   2.6133885   1.2164959  ...  2.5146234  -2.882526
   -0.5533904 ]
  [-0.13708426  1.2616097  -0.1089813  ...  0.4032564  -0.96429455
   -2.2628434 ]]]


In [ ]:
# from google.colab import drive
# drive.mount('/content/drive')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# DATASET_PATH = "/content/drive/MyDrive/caves_open_MVP/"


In [ ]:
import os
# import faiss
import numpy as np
# import torch
import torchvision.transforms as T
from PIL import Image
from tqdm.notebook import tqdm

# Load DINOv2 model
from transformers import AutoImageProcessor, Dinov2Model

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Load the pre-trained DINOv2 model
processor = AutoImageProcessor.from_pretrained("facebook/dinov2-base")
model = Dinov2Model.from_pretrained("facebook/dinov2-base").to(device)
model.eval()

# Define image transformations
transform_image = T.Compose([
    T.ToTensor(),
    T.Resize(244),
    T.CenterCrop(224),
    T.Normalize([0.5], [0.5])
])

def load_image(img_path: str) -> torch.Tensor:
    """Load and transform an image into a tensor for DINOv2."""
    img = Image.open(img_path).convert("RGB")
    return transform_image(img)[:3].unsqueeze(0)

def create_index(dataset_path: str) -> tuple:
    """
    Iterate over all caves and images to create an FAISS index for cave classification.
    """
    index = faiss.IndexFlatL2(768)  # Initialize FAISS index
    image_to_cave = {}  # Mapping from image index to cave name
    all_embeddings = {}

    cave_folders = sorted(os.listdir(dataset_path))  # List all cave folders
    image_id = 0  # Unique index for FAISS

    with torch.no_grad():
        for cave_name in tqdm(cave_folders, desc="Processing Caves"):
            cave_path = os.path.join(dataset_path, cave_name)

            if not os.path.isdir(cave_path):
                continue  # Skip if not a folder

            for img_file in os.listdir(cave_path):
                img_path = os.path.join(cave_path, img_file)


                # Load image and get embedding
                image_tensor = load_image(img_path).to(device)
                embedding = model(image_tensor).last_hidden_state.mean(dim=1).cpu().numpy()

                # Store in FAISS index
                index.add(embedding)
                image_to_cave[image_id] = cave_name  # Map index to cave label
                all_embeddings[img_path] = embedding.tolist()  # Save embedding

                image_id += 1  # Move to next index



    # Save mapping
    # Save JSON with UTF-8 encoding
    with open("image_to_cave.json", "w", encoding="utf-8") as f:
        json.dump(image_to_cave, f, ensure_ascii=False, indent=4)

    with open("all_embeddings.json", "w", encoding="utf-8") as f:
        json.dump(all_embeddings, f, ensure_ascii=False, indent=4)

    faiss.write_index(index, "cave_index.bin")

    return index, image_to_cave

# Run indexing function
data_index, image_to_cave = create_index(DATASET_PATH)


Processing Caves:   0%|          | 0/24 [00:00<?, ?it/s]

In [ ]:

print(image_to_cave)

Processing Caves:   0%|          | 0/26 [00:00<?, ?it/s]

{0: '313ג המלכודת', 1: '313ג המלכודת', 2: '313ג המלכודת', 3: '313ג המלכודת', 4: '354 ערוגות', 5: 'אום קלעה', 6: 'אוצר', 7: 'אצבע', 8: 'אצבע', 9: 'אשלים', 10: 'גולגולת זוטיה', 11: 'גולגולת זוטיה', 12: 'גולגולת זוטיה', 13: 'דיר קדיס', 14: 'דיר קדיס', 15: 'דיר קדיס', 16: 'דיר קדיס', 17: 'דיר קדיס', 18: 'האיגרות', 19: 'האיגרות', 20: 'האמוניט', 21: 'הגיא', 22: 'הגיא', 23: 'הגיא', 24: 'הגיא', 25: 'הנר', 26: 'הנר', 27: 'הנר', 28: 'הנר', 29: 'הנר', 30: 'הנר', 31: 'הנר', 32: 'הנר', 33: 'הנר', 34: 'חתרורים', 35: 'מאשה טרוויל פיטר', 36: 'מאשה טרוויל פיטר', 37: 'מאשה טרוויל פיטר', 38: 'מגלש', 39: 'מגלש', 40: 'מגלש', 41: 'נחל נטוף', 42: 'נמר', 43: 'נמר', 44: 'נקב אוכפים', 45: 'עטרות', 46: 'ערק א- נעסנה', 47: 'ערק א- נעסנה', 48: 'ערק א- נעסנה', 49: 'ערק א- נעסנה', 50: 'ערק א- נעסנה', 51: 'ערק א- נעסנה', 52: 'ערק א- נעסנה', 53: 'קרקס', 54: 'קרקס', 55: 'קרקס', 56: 'שוקבא', 57: 'שוקבא', 58: 'שוקבא', 59: 'שוקבא', 60: 'שן רמון 6', 61: 'שן רמון 6', 62: 'שרך'}


In [ ]:
def search_cave(index, query_image_path, image_to_cave, k=1):
    """Find the most similar cave to the given image."""
    with torch.no_grad():

        query_embedding = model(load_image(query_image_path).to(device)).last_hidden_state.mean(dim=1).cpu().numpy()

    _, indices = index.search(query_embedding, k)  # Search for closest match

    # matched_caves = [image_to_cave[idx] for idx in indices[0]]  # Retrieve cave names

    matched_caves = [image_to_cave[list(image_to_cave.keys())[idx]] for idx in indices[0]]

    return matched_caves

# Example: Query with a new cave image
# query_image = 'DSCN1824.JPG'  # Update with actual image path

# matched_cave = search_cave(data_index, query_image, image_to_cave)
# print(f"The closest matching cave is: {matched_cave[0]}")


In [ ]:
DATASET_PATH = "/content/drive/MyDrive/caves_open_MVP/"
TESTSET_PATH = "/content/drive/MyDrive/cavestesting/"

def evaluate_on_test_set(dataset_index, dataset_image_to_cave, test_index, test_image_to_cave):
    """
    Evaluate the model on the test set by finding the most similar cave from the dataset.
    """
    results = {}
    # print(test_image_to_cave)
    for img_path in test_image_to_cave.items():  # Iterate over actual image paths
        # print(img_path)
        img_path_1 = os.path.join(TESTSET_PATH, img_path[1])
        # print(img_path)
        matched_cave = search_cave(dataset_index, img_path_1, dataset_image_to_cave)
        results[img_path[1]] = matched_cave
        # print(results[img_path[0]])

    with open("test_results.json", "w", encoding="utf-8") as f:
        json.dump(results, f, ensure_ascii=False, indent=4)

    return results

# Example usage
DATASET_PATH = "/content/drive/MyDrive/caves_open_MVP/"
TESTSET_PATH = "/content/drive/MyDrive/cavestesting/"

# Create index from dataset
# data_index, dataset_image_to_cave = create_index(DATASET_PATH)

# Evaluate test images against dataset index
test_results = evaluate_on_test_set(data_index, image_to_cave, test_index,test_image_to_cave)

# Display sample results
for item in test_results.items():
    print(f"Image: {item[0]} -> Closest Match(es): {item[1]}")

Image: masha_tarwill_1.JPG -> Closest Match(es): ['מאשה טרוויל פיטר']
Image: hamalkodet_1.jpg -> Closest Match(es): ['313ג המלכודת']
Image: hamalkodet_2.jpg -> Closest Match(es): ['313ג המלכודת']
Image: Um_kalaa_2.jpg -> Closest Match(es): ['נמר']
Image: otzar_1.JPG -> Closest Match(es): ['שוקבא']
Image: arugot_1.jpg -> Closest Match(es): ['שן רמון 6']
Image: etzba_1.jpg -> Closest Match(es): ['אצבע']
Image: ashalim_1.JPG -> Closest Match(es): ['אשלים']
Image: golgolet_1.JPG -> Closest Match(es): ['הגיא']
Image: golgolet_2.JPG -> Closest Match(es): ['הגיא']
Image: dir_kadis_1.JPG -> Closest Match(es): ['דיר קדיס']
Image: haigrot_1.jpg -> Closest Match(es): ['313ג המלכודת']
Image: haamonit_1.JPG -> Closest Match(es): ['האמוניט']
Image: hagai_1.jpg -> Closest Match(es): ['הגיא']
Image: haner_1.JPG -> Closest Match(es): ['הנר']
Image: chatrurim_1.JPG -> Closest Match(es): ['חתרורים']
Image: miglash_1.JPG -> Closest Match(es): ['הגיא']
Image: nahal_natuf_1.JPG -> Closest Match(es): ['נחל נ